In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import multiprocessing
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from multiprocessing import Pool

from selenium.common.exceptions import TimeoutException

import pickle

In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path("../lib/").absolute()))

In [7]:
def login():
    with open('credentials.txt', 'r') as f:
        password = f.readline()

    driver = webdriver.Firefox()
    driver.implicitly_wait(10)
    driver.get("https://www.scribophile.com/dashboard/login")
    driver.find_element_by_name('email').send_keys('seokho_hong@yahoo.com')
    driver.find_element_by_name('password').send_keys(password)
    driver.find_element_by_name('password').send_keys(Keys.RETURN)
    return driver
    
def scrape_page(driver, page_num, text_results):
    wait = WebDriverWait(driver, 10)
    wait.until(EC.element_to_be_clickable((By.ID, 'feed')))
    driver.get('https://www.scribophile.com/writing/browse?page=' + str(page_num) + '&genre=fiction&type=novel&showrestrictedwork=false&showlockedwork=false&showpersonalspotlightwork=false&showkarmafreespotlightwork=false&query=&tag=')
    # get the works on the page
    links = []
    for elem in driver.find_elements_by_class_name('work-details'):
        links.append(elem.find_element_by_tag_name('a').get_attribute('href'))
        
    for link in links:
        try:
            driver.get(link)
            driver.find_element_by_id('work-body').find_element_by_class_name('pill-button').click()
            text_results[link] = driver.find_element_by_id('work-body').text
        except TimeoutException:
            pass
    driver.quit()

def run_scraper(page_num, text_results):
    driver = login()
    scrape_page(driver, page_num, text_results)

In [8]:
def scrape_pool(page_range, num_processes=2):
    pool = Pool(num_processes)
    manager = multiprocessing.Manager()
    text_results = manager.dict()
    jobs = []
    for i in page_range:
        pool.apply_async(run_scraper, args=(i, text_results))

    pool.close()
    pool.join()

    return text_results

In [9]:
increment_size = 20
for corpus_index in range(10):
    texts = scrape_pool(range(corpus_index * increment_size, (corpus_index + 1) * increment_size), num_processes=5)
    with open('scribophile/scribophile_texts' + str(corpus_index) + '.txt', 'w') as f:
        f.writelines(texts.values())
    #with open('scribophile_texts' + str(corpus_index) + ".pkl", 'wb') as f:
    #    pickle.dump(texts.items(), f)

Process ForkPoolWorker-30:
Process ForkPoolWorker-34:
Process ForkPoolWorker-32:
Process ForkPoolWorker-31:
Process ForkPoolWorker-33:
Traceback (most recent call last):


KeyboardInterrupt: 

  File "/Users/sehong/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/sehong/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/sehong/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/sehong/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
  File "/Users/sehong/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
  File "<ipython-input-7-46d1c1b58b74>", line 32, in run_scraper
    driver = login()
  File "/Users/sehong/anaconda3/lib/python3.7/si

  File "/Users/sehong/anaconda3/lib/python3.7/site-packages/selenium/webdriver/remote/remote_connection.py", line 397, in _request
    resp = self._conn.request(method, url, body=body, headers=headers)
  File "/Users/sehong/anaconda3/lib/python3.7/site-packages/urllib3/request.py", line 68, in request
    **urlopen_kw)
  File "/Users/sehong/anaconda3/lib/python3.7/site-packages/urllib3/request.py", line 89, in request_encode_url
    return self.urlopen(method, url, **extra_kw)
  File "/Users/sehong/anaconda3/lib/python3.7/site-packages/urllib3/request.py", line 89, in request_encode_url
    return self.urlopen(method, url, **extra_kw)
  File "/Users/sehong/anaconda3/lib/python3.7/site-packages/urllib3/request.py", line 68, in request
    **urlopen_kw)
  File "/Users/sehong/anaconda3/lib/python3.7/site-packages/urllib3/request.py", line 89, in request_encode_url
    return self.urlopen(method, url, **extra_kw)
  File "/Users/sehong/anaconda3/lib/python3.7/site-packages/urllib3/poolmanag